In [ ]:
#Making a new "dataset" with segmented MiniKinetics50 videos

#1. Go through each video of dataset/mini50_clean_train.csv
#2. Run YOLO + segmentation on it
#3. If it could not find a person:
    # Continue
#4. If it did find a person:
    # a) Add the video frames to the directory I just made
        # in dataset/segmented_minikinetics50/train, create (if not created yet)
        # or go to the directory corresponding to the label
            # Create a new subdirectory with the youtube id only
            # Add in the segmented images
    # b) Write the row into the new csv

import os
import pandas as pd
import tempfile
import shutil
import numpy as np
import torch
import sys

os.chdir("/n/fs/visualai-scr/temp_LLP/ellie/slowfast_kinetics/sam2")
sys.path.insert(0, "/n/fs/visualai-scr/temp_LLP/ellie/slowfast_kinetics/sam2")
# print("current working direcotry", os.getcwd())
from sam2.build_sam import build_sam2_video_predictor
os.chdir("/n/fs/visualai-scr/temp_LLP/ellie/slowfast_kinetics")

In [11]:
yolo = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)

sam2_checkpoint = "/n/fs/visualai-scr/temp_LLP/ellie/slowfast_kinetics/sam2/checkpoints/sam2.1_hiera_large.pt"

#model_cfg, expects a config path, not an absolute path
os.chdir("/n/fs/visualai-scr/temp_LLP/ellie/slowfast_kinetics/sam2/sam2")
model_cfg = "configs/sam2.1/sam2.1_hiera_l.yaml"

predictor = build_sam2_video_predictor(model_cfg, sam2_checkpoint, device='cpu')
#TODO: reduce df to the max_videos if specified

Using cache found in /u/ez9517/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2025-7-1 Python-3.10.13 torch-2.6.0+cu124 CUDA:0 (NVIDIA L40, 45488MiB)

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


In [12]:
device = 'cpu'

In [4]:
df = pd.read_csv("dataset/mini50_clean_train.csv")
output_root = "dataset/segmented_minikinetics50/train"
print(df.head())

                                 label   youtube_id  time_start  time_end  \
0                 dribbling basketball  --4-0ihtnBU          58        68   
1                       playing tennis  --56QUhyDQM         185       195   
2                      climbing a rope  --EaS9P7ZdQ          13        23   
3                       brushing teeth  --IPbe5ZMCI           2        12   
4  skiing (not slalom or crosscountry)  --TBx-Spzis         148       158   

   split                                          full_path  num_files  
0  train  /n/fs/visualai-scr/Data/Kinetics_cvf/frames/tr...        113  
1  train  /n/fs/visualai-scr/Data/Kinetics_cvf/frames/tr...        300  
2  train  /n/fs/visualai-scr/Data/Kinetics_cvf/frames/tr...        300  
3  train  /n/fs/visualai-scr/Data/Kinetics_cvf/frames/tr...        300  
4  train  /n/fs/visualai-scr/Data/Kinetics_cvf/frames/tr...        300  


In [ ]:
# Compute indices for 8 and 32 evenly spaced frames
def sample_indices(self, n, total_frames):
    return [int(round(i * (total_frames - 1) / (n - 1) + 1)) for i in range(n)]

In [ ]:
#Given a row in csv, sample indices, run yolo & video segmentation on it
def run_yolo_and_seg(row):
    segmented_frames = [] #A list of PIL images containing all the segmented frames

    video_path = row['full_path']
    label = row['label']
    youtube_id = row['youtube_idx']
    num_files = row['num_files']
    time_start = row['time_start']
    time_end = row['time_end']
    indices = sample_indices(32, num_files)
    hasPerson = False


    #Make a temporary directory storing video frames at certain indices
    #Make a numpy array "frames" to store the video frames at certain indices
    temp_dir = tempfile.mkdtemp()
    frames = []
    for i in indices:
        source = os.path.join(video_path, f"{i:06d}.jpg")
        dest = os.path.join(temp_dir, f"{i:06d}.jpg")
        os.symlink(source, dest)

        img = Image.open(source).convert('RGB')  # Load as RGB
        img_np = np.array(img) #convert to np image
        frames.append(img_np)

    video_np = np.stack(frames, axis=0)
    print("VIDEO NP shape", video_np.shape)

    img0_np = video_np[0]
    results = yolo(img0_np) #get bbox for the first frame
    bboxes = results.xyxy[0]
    person_bboxes = bboxes[bboxes[:, 5] == 0] # get bboxes for class == person only
    person_bboxes = person_bboxes[:, :4].cpu().numpy()
    
    inference_state = predictor.init_state(video_path=temp_dir)

    #If a person was detected, then segment out the person
    if person_bboxes.shape[0] >= 1:
        hasPerson = True
        box = np.array(person_bboxes[0], dtype=np.float32)
        _, out_obj_ids, out_mask_logits = predictor.add_new_points_or_box(
            inference_state=inference_state,
            frame_idx=0,
            obj_id=1,
            box=box,
        )

        #Go through each video frame, propogate the segmentation, calculate the binary mask
        for out_frame_idx, _, out_mask_logits in predictor.propagate_in_video(inference_state):
            binarymask = (out_mask_logits.squeeze(0).squeeze(0)) > 0.0 #binary mask, where 1 is person, 0 isnot
            binarymask = binarymask.unsqueeze(2).to(device)

            img = torch.from_numpy(video_np[out_frame_idx]).to(device)
            seg_img = binarymask * img
            seg_img_np = seg_img.cpu().numpy()
            if seg_img_np.shape[0] == 3:  # (3, H, W) -> (H, W, 3)
                seg_img_np = np.transpose(seg_img_np, (1, 2, 0))

            segmented_frames.append(seg_img_np)

    return segmented_frames, hasPerson         

In [ ]:
            #Save the segmented image to the new dataset directory path
            label_path = f"/n/fs/visualai-scr/temp_LLP/ellie/slowfast_kinetics/dataset/segmented_minikinetics50/train/{label}"
            os.makedirs(label_path, exist_ok=True)
            vid_path = os.makedirs(os.join(label_path, f"{youtube_id}_{time_start:06d}_{time_end:06d}"))
            os.makedirs(vid_path, exist_ok=True)
            Image.fromarray(seg_img_np).save(os.join(vid_path, f"{out_frame_idx:06d}.jpg"))   

In [ ]:
for idx, row in df.iterrows():
    video_path = row['full_path']
    label = row['label']
    youtube_id = row['youtube_idx']
    num_files = row['num_files']

    hasPerson = run_seg_and_save(row)

    if hasPerson == True:
        

    
    
    